# SARAH Toolbox
## Collection of models to be run from a docker image
Launch virtual docker image: 
[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/SM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)
## Implemented models
Each model is to be run in a specific virtual machine. Follow the binder button in each github repo
* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/SM/HighScale` (For RGE running: ./butler SM/HighScale) [Repo](https://github.com/restrepo/SM-Toolbox/)
* `SARAH/Models/diphotonModels/LRmodels` (Not public yet)
## Instructions to compile the model
In `SPHENO` and `micromegas`

See possible analysis based on the models in  [./tests](./tests) folder.

Below we define the model to be compiled:

In [14]:
MODEL=SMHighScale

SMHighScale


It is recomended that the commands be executed [from a terminal](../terminals/1).

Be sure that `SPHENO/Makefile` have `gfortan` uncommented:

In [1]:
cat  SPHENO/Makefile | grep "F90 ="

# F90 = nagfor
F90 = gfortran
# F90 = g95
# F90 = lf95
#F90 = ifort


In [7]:
./compile_spheno_directly.sh $MODEL

make: Entering directory '/home/main/notebooks/SPHENO'
cd /home/main/notebooks/SPHENO/SMHighScale ; make F90=gfortran version=330.00
make[1]: Entering directory '/home/main/notebooks/SPHENO/SMHighScale'
cd ../src ; make F90=gfortran 
make[2]: Entering directory '/home/main/notebooks/SPHENO/src'
gfortran -c -O -J../include -I../include  -DGENERATIONMIXING  -DONLYDOUBLE Control.F90
ar -ruc ../lib/libSPheno.a Control.o
rm -f Control.o
gfortran -c -O -J../include -I../include Mathematics.f90
ar -ruc ../lib/libSPheno.a Mathematics.o
rm -f Mathematics.o
gfortran -c -O -J../include -I../include  -DGENERATIONMIXING  -DONLYDOUBLE RGEs.F90
RGEs.F90:98:0: warning: extra tokens at end of #endif directive
 # endif SEESAWIII
 ^
RGEs.F90:14740:0: warning: extra tokens at end of #endif directive
 #endif SEESAWIII
 ^
RGEs.F90:19645:0: warning: extra tokens at end of #endif directive
 #endif SEESAWIII
 ^
ar -ruc ../lib/libSPheno.a RGEs.o
rm -f RGEs.o
gfortran -c -O -J../include -I../include MathematicsQ

make a work dir

In [8]:
mkdir -p work
cd work

If neccesary, turn ON tachyonic spectrum with `52 1 #...`

In [10]:
cat ../SPHENO/$MODEL/LesHouches.in.$MODEL | grep 'Write spectrum in case of tachyonic states' 

 52 0               # Write spectrum in case of tachyonic states 


In [11]:
../SPHENO/bin/SPheno$MODEL ../SPHENO/$MODEL/LesHouches.in.$MODEL

 Calculating mass spectrum
              1 .-iteration
   ... reached precision:   15.171277662106421     
              2 .-iteration
   ... reached precision:   3.5838925311539441E-002
              3 .-iteration
   ... reached precision:   1.7529214536961314E-004
              4 .-iteration
   ... reached precision:   9.1119403882726111E-007
 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


Compile `micromegas` model

In [12]:
cd ../micromegas
make
make
cd $MODEL
make main=CalcOmega.cpp

echo \#define micrO \"/home/main/notebooks/micromegas\"  > sources/microPath.h
make -C CalcHEP_src
make[1]: Entering directory '/home/main/notebooks/micromegas/CalcHEP_src'
./getFlags
MAKE= make
UNIX Linux
C compiler detected
can not generate shared library
tring to add -fPIC option
: shared liblary generation OK
Trying to detect X11
search by xmkmf
      fails
search X11 according to list of possible distinations
      fails
X11 not detected. CalcHEP will be compiled in Blind mode
Fortran compiler is detected
gfortran OK
FFLAGS OK
./sbin/setPath /home/main/notebooks/micromegas/CalcHEP_src
chmod 755 mkWORKdir
make -C c_source
make[2]: Entering directory '/home/main/notebooks/micromegas/CalcHEP_src/c_source'
make -C getmem 
make[3]: Entering directory '/home/main/notebooks/micromegas/CalcHEP_src/c_source/getmem'
gcc -g -fsigned-char -Wstrict-prototypes -Wimplicit -Wmissing-prototypes -std=gnu99 -fPIC -Iinclude -I../chep_crt/include   -c -o getmem.o getmem.c
ar rv ../../lib/serv.a getmem

Run micromegas

In [13]:
if [ "$MODEL" != SMHighScale ]; then
    ../micromegas/$MODEL/CalcOmega SPheno.spc.$MODEL
fi